In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.stats import norm
pd.set_option('display.max_colwidth', None)
from scipy.optimize import minimize
from scipy.optimize import fsolve
from scipy.integrate import quad
import scipy.integrate as integrate
import scipy.special as special
import math
import random
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import quantecon as qe
from mpl_toolkits.mplot3d import Axes3D

https://stackoverflow.com/questions/8739227/how-to-solve-a-pair-of-nonlinear-equations-using-python https://stackoverflow.com/questions/41687908/python-nsolve-solve-triple-of-equations

|        |          |      | From/Start |       |
|--------|----------|------|------------|-------|
|        |          | Mild | Moderate   | Harsh |
|        | Mild     | 0.7  | 0.2        | 0.1   |
| To/End | Moderate | 0.2  | 0.7        | 0.2   |
|        | Harsh    | 0.1  | 0.1        | 0.7   |

In [8]:
# The statespace
states = ["Mild","Moderate","Harsh"]

# Possible sequences of events
transitionName = [["LL","LM","LH"],["ML","MM","MH"],["HL","HM","HH"]]

# Probabilities matrix (transition matrix)
transitionMatrix = [[0.7,0.2,0.1],[0.1,0.7,0.2],[0.2,0.1,0.7]]

In [9]:
if sum(transitionMatrix[0])+sum(transitionMatrix[1])+sum(transitionMatrix[1]) != 3:
    print("Somewhere, something went wrong. Transition matrix, perhaps?")
else: print("All is gonna be okay, you should move on!! ;)")

All is gonna be okay, you should move on!! ;)


In [7]:
transitionMatrix[1]

[0.2, 0.7, 0.1]

In [14]:
ψ = (0.3, 0.7)           # probabilities over {0, 1}
cdf = np.cumsum(ψ)       # convert into cummulative distribution
qe.random.draw(cdf, 5)   # generate 5 independent draws from ψ

array([1, 1, 1, 1, 1])

In [15]:
def mc_sample_path(P, ψ_0=None, sample_size=1_000):

    # set up
    P = np.asarray(P)
    X = np.empty(sample_size, dtype=int)

    # Convert each row of P into a cdf
    n = len(P)
    P_dist = [np.cumsum(P[i, :]) for i in range(n)]

    # draw initial state, defaulting to 0
    if ψ_0 is not None:
        X_0 = qe.random.draw(np.cumsum(ψ_0))
    else:
        X_0 = 0

    # simulate
    X[0] = X_0
    for t in range(sample_size - 1):
        X[t+1] = qe.random.draw(P_dist[X[t]])

    return X

In [16]:
P = [[0.4, 0.6],
     [0.2, 0.8]]

In [17]:
X = mc_sample_path(P, ψ_0=[0.1, 0.9], sample_size=100_000)
np.mean(X == 0)

0.2473

In [18]:
X

array([1, 0, 0, ..., 1, 1, 1])

In [19]:
1_000

1000

In [20]:
from quantecon import MarkovChain

mc = qe.MarkovChain(P)
X = mc.simulate(ts_length=1_000_000)
np.mean(X == 0)

0.250146

In [21]:
X

array([1, 0, 1, ..., 1, 1, 1])

In [22]:
mc = qe.MarkovChain(P, state_values=('unemployed', 'employed'))
mc.simulate(ts_length=4, init='employed')

array(['employed', 'employed', 'employed', 'employed'], dtype='<U10')

In [23]:
mc.simulate(ts_length=4, init='unemployed')

array(['unemployed', 'employed', 'employed', 'employed'], dtype='<U10')

In [24]:
mc.simulate(ts_length=4)  # Start at randomly chosen initial state

array(['employed', 'employed', 'employed', 'employed'], dtype='<U10')

In [ ]:
def brownian(x0, n, dt, delta, mu, out=None):
    """
    Generate an instance of Brownian motion (i.e. the Wiener process):

        X(t) = X(0) + N(0, delta**2 * t; 0, t)

    where N(a,b; t0, t1) is a normally distributed random variable with mean a and
    variance b.  The parameters t0 and t1 make explicit the statistical
    independence of N on different time intervals; that is, if [t0, t1) and
    [t2, t3) are disjoint intervals, then N(a, b; t0, t1) and N(a, b; t2, t3)
    are independent.
    
    Written as an iteration scheme,

        X(t + dt) = X(t) + N(0, delta**2 * dt; t, t+dt)


    If `x0` is an array (or array-like), each value in `x0` is treated as
    an initial condition, and the value returned is a numpy array with one
    more dimension than `x0`.

    Arguments
    ---------
    x0 : float or numpy array (or something that can be converted to a numpy array
         using numpy.asarray(x0)).
        The initial condition(s) (i.e. position(s)) of the Brownian motion.
    n : int
        The number of steps to take.
    dt : float
        The time step.
    delta : float
        delta determines the "speed" of the Brownian motion.  The random variable
        of the position at time t, X(t), has a normal distribution whose mean is
        the position at time t=0 and whose variance is delta**2*t.
    out : numpy array or None
        If `out` is not None, it specifies the array in which to put the
        result.  If `out` is None, a new numpy array is created and returned.

    Returns
    -------
    A numpy array of floats with shape `x0.shape + (n,)`.
    
    Note that the initial value `x0` is not included in the returned array.
    """

    x0 = np.asarray(x0)

    # For each element of x0, generate a sample of n numbers from a
    # normal distribution.
    r = norm.rvs(size=x0.shape + (n,), scale=delta*sqrt(dt))

    # If `out` was not given, create an output array.
    if out is None:
        out = np.empty(r.shape)

    # This computes the Brownian motion by forming the cumulative sum of
    # the random samples. 
    np.cumsum(r, axis=-1, out=out)

#     out = out + 1000
    # Add the initial condition.
    out += np.expand_dims(x0, axis=-1)
    out = out + np.repeat(np.arange(n).reshape(1,-1),x0.shape[0],axis=0)*mu/n
    print(out)

    return out
# x1 = brownian(x[:,0], N, dt, delta, mu)
# x[:,1:] = x1